# Installation

In [ ]:
!pip install -q transformers trl peft accelerate bitsandbytes
!pip install --upgrade transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 93.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig , GemmaTokenizerFast

# Model Loading


In [ ]:
from google.colab import userdata

os.environ['HUGGINGFACE_TOKEN'] = userdata.get('HUGGINGFACE_TOKEN')
model_id = "google/gemma-2b"


In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token = os.environ['HUGGINGFACE_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, token = os.environ['HUGGINGFACE_TOKEN'])

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
text = "inputs :Can you give me interview questions about Angular?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
os.environ['WANDB_PROJECT'] = "false"


inputs :Can you give me interview questions about Angular?

Answer:

1. What is Angular?
Angular is a JavaScript framework that is used to


# LoRA Configuration

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj" , "up_proj" , "down_proj"],
    task_type = "CAUSAL_LM"
)



# Dataset Preparation

In [ ]:
data = load_dataset("ali-alkhars/interviews")


README.md: 0.00B [00:00, ?B/s]

interviews_dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2292 [00:00<?, ? examples/s]

In [ ]:
def formating(example):
  text = f"Quote:{example['input']}\nrsponce :{example['response']}"
  return text

# Training

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = data["train"],
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 2,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        output_dir = "outputs",
        optim = "paged_adamw_8bit"
    ),
    peft_config = lora_config,
    formatting_func = formating,
)

Applying formatting function to train dataset:   0%|          | 0/2292 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2292 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2292 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2292 [00:00<?, ? examples/s]

In [11]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shreyshukla512 (shreyshukla512-a-d-patel-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,5.053300
2,4.362200
3,4.418400
4,4.721200
5,3.589000
6,3.149900
7,3.048500
8,2.644800
9,2.282300
10,2.415400


/usr/local/lib/python3.12/dist-packages/peft/utils/other.py:1228: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-68dd8278-015f832f2ebd9bc04cfe9ad0;53f5aee7-c133-4fe0-b55d-fb0f8c54884b)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Access to model google/gemma-2b is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2b.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:286: UserWarning: Could not find a config file in google/gemma-2b - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=100, training_loss=1.674757564663887, metrics={'train_runtime': 309.8294, 'train_samples_per_second': 1.291, 'train_steps_per_second': 0.323, 'total_flos': 124771426775040.0, 'train_loss': 1.674757564663887, 'epoch': 0.17452006980802792})

In [13]:
text = "input : interview questions about java?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


input : interview questions about java?
rsponce : What is the difference between a constructor and a factory method?




# Model Saving


In [14]:
!pip install -q huggingface_hub

from huggingface_hub import login
login()  # paste your HF token in the Colab prompt (secure); do not hardcode in code

In [15]:
repo_id = "shrey318/gemma2b-qlora-adapters"  # your repo

# Creates repo if it doesn't exist; set private if you want
trainer.model.push_to_hub(repo_id, private=True)
tokenizer.push_to_hub(repo_id)
print("Pushed adapters + tokenizer to:", repo_id)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 11.3kB / 39.3MB            

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...p21cy5qc0/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...mp21cy5qc0/tokenizer.json:  19%|#8        | 6.46MB / 34.4MB            

Pushed adapters + tokenizer to: shrey318/gemma2b-qlora-adapter


# Testing

In [16]:


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE_ID = "google/gemma-2b"
ADAPTERS_REPO = "shrey318/gemma2b-qlora-adapters"  # your repo

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(BASE_ID)
base_model = AutoModelForCausalLM.from_pretrained(BASE_ID, quantization_config=bnb)

model = PeftModel.from_pretrained(base_model, ADAPTERS_REPO)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linea

In [17]:
prompt = "js question?."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

print(tokenizer.decode(out[0], skip_special_tokens=True))

js question?.
rsponce :What are the common ways to implement a real-time messaging system?
